In [1]:
import requests

# smoke-test -> have you started the API?
requests.get("http://localhost:31476/docs")

<Response [200]>

In [2]:
t = requests.post(
    "http://localhost:31476/token", data={"username": "nenb", "password": "nenb"}
).json()

headers = {"Authorization": f"Bearer {t}"}

In [3]:
# there is a new filesystems component
requests.get("http://localhost:31476/components", headers=headers).json()

{'filesystems': ['github', 'local'],
 'documents': ['.pdf', '.txt'],
 'source_storages': [{'properties': {'chunk_overlap': {'default': 250,
     'title': 'Chunk Overlap',
     'type': 'integer'},
    'chunk_size': {'default': 500, 'title': 'Chunk Size', 'type': 'integer'},
    'num_tokens': {'default': 1024, 'title': 'Num Tokens', 'type': 'integer'}},
   'required': [],
   'title': 'Chroma',
   'type': 'object'},
  {'properties': {'chunk_overlap': {'default': 250,
     'title': 'Chunk Overlap',
     'type': 'integer'},
    'chunk_size': {'default': 500, 'title': 'Chunk Size', 'type': 'integer'},
    'num_tokens': {'default': 1024, 'title': 'Num Tokens', 'type': 'integer'}},
   'required': [],
   'title': 'LanceDB',
   'type': 'object'}],
 'assistants': [{'properties': {'max_new_tokens': {'default': 256,
     'title': 'Max New Tokens',
     'type': 'integer'}},
   'title': 'OpenAI/gpt-4',
   'type': 'object'},
  {'properties': {'max_new_tokens': {'default': 256,
     'title': 'Max New T

In [4]:
name = "bm25.pdf"
prefixed_path = (
    "github://papers-we-love/papers-we-love/information_retrieval/okapi-at-trec3.pdf"
)

# this creates the relevant metadata on the server and stores in the db
metadata = requests.get(
    f"http://localhost:31476/document?name={name}&prefixed_path={prefixed_path}",
    headers=headers,
).json()
docs = metadata["document"]

In [5]:
import json

data = json.dumps(
    {
        "name": "BM25",
        "source_storage": "LanceDB",
        "assistant": "OpenAI/gpt-3.5-turbo-16k",
        "params": {},
        "documents": [docs],
    }
)

In [6]:
# create a new chat
chat_metadata = requests.post(
    "http://localhost:31476/chats", headers=headers, data=data
).json()
chat_id = chat_metadata["id"]

In [7]:
# this retrieves the pdf from github, and stores the embeddings on the server
requests.post(f"http://localhost:31476/chats/{chat_id}/prepare", headers=headers).json()

In [8]:
prompt = "What were the new developments for TREC-3?"
response = requests.post(
    f"http://localhost:31476/chats/{chat_id}/answer?prompt={prompt}", headers=headers
).json()

print(response["message"]["content"])

The new developments for TREC-3 included the introduction of the simple inverse collection frequency (ICF) term-weighting scheme, which incorporated within-document frequency, document length, and within-query frequency components. Additionally, there were advancements in automatic ad hoc and routing results, as well as the development of a user interface and search framework. Query expansion and routing term selection were also successful developments. Modified term-weighting functions and passage retrieval had small beneficial effects.
